### Import relevant libraries

In [9]:
from semopy import Model, calc_stats
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
import pandas as pd
import numpy as np
import seaborn as sns
from semopy.inspector import inspect
# from semopy.diagram import write_png
# from semopy import build_model_diagram


### Import the dataset

In [21]:
df = pd.read_csv("Welzijnsmonitor2025_scaled_normalised_UTF8.csv", sep=";")

### Creating the model

In [17]:
model_desc = """
# Measurement model

Engagement_and_Burnout_1 =~ Bevl_1 + Bevl_2 + Bevl_3
Behavior_1 =~ Cogn_Eng1_5 + Cogn_Eng1_6 + Cogn_Eng2_2 + Cogn_Eng2_3 + Cogn_Eng2_8
Motivation =~ Mot_Stress_1 + Mot_Stress_4
Resilience_1 =~ Veer_1 + Veer_5
Wellbeing =~ Engagement_and_Burnout_1 + Behavior_1 + Motivation + Resilience_1

Knowledge_of_Interventions =~ Bekendgebruik_1 + Bekendgebruik_2 + Bekendgebruik_3 + Bekendgebruik_4 + Bekendgebruik_5 + Bekendgebruik_6 + Bekendgebruik_7

Background_Features_1 =~ Betrok_Ouders

Behavior_2 =~ Cogn_Eng1_1 + Cogn_Eng1_2 + Cogn_Eng1_3 + Cogn_Eng1_4 + Cogn_Eng2_1 + Cogn_Eng2_4 + Cogn_Eng2_5 + Cogn_Eng2_6 + Cogn_Eng2_7

Extra_Engagement =~ Cogregiedocent_1 + Cogregiedocent_2 + Cogregiedocent_3 +  Cogregiedocent_4 + Cogregiedocent_5 + Cogregiedocent_6 + Extr_eng2_1 + Extr_eng2_2 + Extr_eng2_3

Need_for_Help =~ Hulp_1 + Hulp_2 + Hulp_3
Participation =~ Partici1_1 + Partici1_2 + Partici1_3 + Partici1_4

Energy_Sources =~ Knowledge_of_Interventions + Background_Features_1 + Behavior_2 + Extra_Engagement + Need_for_Help + Participation

Negative_Outcomes =~ StopInt + Vertr

Study_Performance =~ Cijfer_huidig_1 + StPunt_beh
Background_Features_2 =~ Stopint2
Positive_Outcomes =~ Study_Performance + Background_Features_2

Lifestyle =~ Cantrill_1 + Leefst + Q297_1

Stress_1 =~ Onnodige_stress_1 + Onnodige_stress_2 + Onnodige_stress_3 + Onnodige_stress_4 + Onnodige_stress_5 + Onnodige_stress_6 + Onnodige_stress_7 + Onnodige_stress_8 + Onnodige_stress_9 + Onnodige_stress_10

Pressure_to_Perform =~ Pres_1 + Pres_2 + Pres_12
Background_Features_3 =~ Werk_1

Anxiety =~ Zorg_1 + Zorg_2 + Zorg_3 + Zorg_4 + Zorg_5 + Zorg_6

Stressors =~ Lifestyle + Stress_1 + Pressure_to_Perform + Background_Features_3 + Anxiety

Engagement_and_Burnout_2 =~ Burn1_1 + Burn1_2 + Burn1_3 + Burn1_4
Depression =~ Depr_1 + Depr_2 + Depr_3 + Depr_4 + Depr_5 + Depr_6 + Depr_7 + Depr_8

Stress_2 =~ Mot_Stress_2
Resilience_2 =~ Veer_2 + Veer_3 + Veer_4 + Veer_6

Response_to_Stress =~ Engagement_and_Burnout_2 + Depression + Stress_2 + Resilience_2
"""

In [18]:
model = Model(model_desc)
model.fit(df)
latent_scores = model.predict_factors(df)

LinAlgError: Singular matrix

# Export the model with the new latent variables

In [22]:
latent_scores = model.predict_factors(df)

latent_scores.to_csv("latent_variable_scores.csv", index=False)

print("Latent variable scores extracted:")
print(latent_scores.head())

LinAlgError: Singular matrix

### Create a minmax normalization for latent variables (Temporary solution)

In [ ]:
scaler = MinMaxScaler()

latent_scores_normalized = pd.DataFrame(
    scaler.fit_transform(latent_scores),
    columns=latent_scores.columns,
    index=latent_scores.index
)

In [ ]:
df_with_latent_norm = df.copy()
for col in latent_scores_normalized.columns:
    df_with_latent_norm[col + "_norm"] = latent_scores_normalized[col]

# Save if needed
df_with_latent_norm.to_csv("data_with_normalized_latent_variables.csv", index=False)

### Calculate fit indices

In [ ]:
stats = calc_stats(model)

print("=== Model Fit Statistics ===")
print(stats.T)

=== Model Fit Statistics ===
                      Value
DoF             3879.000000
DoF Baseline    4005.000000
chi2           21676.237683
chi2 p-value       0.000000
chi2 Baseline  56457.934823
CFI                0.660701
GFI                0.616064
AGFI               0.603593
NFI                0.616064
TLI                0.649680
RMSEA              0.055086
AIC              403.346679
BIC             1552.866217
LogLik            14.326661


### Create and save the path diagram
Doesn't work yet

In [ ]:
# Produce a Graphviz object
g = build_model_diagram(model)

# Export diagram
g.render("sem_path_diagram", format="png", cleanup=True)

print("Diagram saved.")

In [ ]:
# regressions
# Stressors ~ Energy_Sources
# Energy_Sources ~ Stressors

# Response_to_Stress ~ Stressors + Energy_Sources + Wellbeing
# Wellbeing ~ Stressors + Energy_Sources + Response_to_Stress

# Positive_Outcomes ~ Wellbeing + Response_to_Stress
# Negative_Outcomes ~ Wellbeing + Response_to_Stress